<a href="https://colab.research.google.com/github/rlevy820/scotus-research/blob/main/DecisionTree02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn import tree
from sklearn.model_selection import train_test_split
# decision tree visualization
from sklearn.tree import export_graphviz
import six
import sys
sys.modules['sklearn.externals.six'] = six
from six import StringIO
from IPython.display import Image
import pydotplus

In [20]:
df = pd.read_csv('https://raw.githubusercontent.com/rlevy820/scotus-research/refs/heads/main/data/Case%20Centered%20Data.csv')
df = df[df['partyWinning'] != 2]
df.head()

,caseId,docketId,caseIssuesId,voteId,dateDecision,decisionType,usCite,sctCite,ledCite,lexisCite,...,authorityDecision1,authorityDecision2,lawType,lawSupp,lawMinor,majOpinWriter,majOpinAssigner,splitVote,majVotes,minVotes
0,1946-001,1946-001-01,1946-001-01-01,1946-001-01-01-01,11/18/1946,1,329 U.S. 1,67 S. Ct. 6,91 L. Ed. 3,1946 U.S. LEXIS 1724,...,4.0,NaN,6.0,600.0,35 U.S.C. § 33,78.0,78.0,1,8,1
1,1946-002,1946-002-01,1946-002-01-01,1946-002-01-01-01,11/18/1946,1,329 U.S. 14,67 S. Ct. 13,91 L. Ed. 12,1946 U.S. LEXIS 1725,...,4.0,NaN,6.0,600.0,18 U.S.C. § 398,81.0,87.0,1,6,3
2,1946-002,1946-002-02,1946-002-02-01,1946-002-02-01-01,11/18/1946,1,329 U.S. 14,67 S. Ct. 13,91 L. Ed. 12,1946 U.S. LEXIS 1725,...,4.0,NaN,6.0,600.0,18 U.S.C. § 398,81.0,87.0,1,6,3
3,1946-002,1946-002-03,1946-002-03-01,1946-002-03-01-01,11/18/1946,1,329 U.S. 14,67 S. Ct. 13,91 L. Ed. 12,1946 U.S. LEXIS 1725,...,4.0,NaN,6.0,600.0,18 U.S.C. § 398,81.0,87.0,1,6,3
4,1946-002,1946-002-04,1946-002-04-01,1946-002-04-01-01,11/18/1946,1,329 U.S. 14,67 S. Ct. 13,91 L. Ed. 12,1946 U.S. LEXIS 1725,...,4.0,NaN,6.0,600.0,18 U.S.C. § 398,81.0,87.0,1,6,3


In [32]:
data1 = df[['term', 'caseOrigin', 'issueArea', 'petitioner', 'respondent', 'lcDispositionDirection', 'declarationUncon', 'partyWinning']]
data2 = data1.dropna()

inputs = data2[['term', 'caseOrigin', 'issueArea', 'petitioner', 'respondent', 'lcDispositionDirection', 'declarationUncon']]
target = data2['partyWinning']

In [34]:
X_train, X_test, y_train, y_test = train_test_split(inputs, target, test_size=0.2)

In [35]:
model = tree.DecisionTreeClassifier()
model.fit(X_train, y_train)

DecisionTreeClassifier()

In [36]:
model.score(X_test, y_test)

0.749811320754717

In [38]:
# prompt: create a UI that allows one to select input features and it gives the models prediction, make them dropdowns for the possible values, if predict=0 print denied, if 1 print affimed, clear prediction result after each button press

import ipywidgets as widgets
from IPython.display import display, clear_output
import warnings
warnings.filterwarnings('ignore')

# Create dropdown widgets for each input feature
term_widget = widgets.Dropdown(options=sorted(df['term'].unique()), description='Term:')
caseOrigin_widget = widgets.Dropdown(options=sorted(df['caseOrigin'].unique()), description='Case Origin:')
issueArea_widget = widgets.Dropdown(options=sorted(df['issueArea'].unique()), description='Issue Area:')
petitioner_widget = widgets.Dropdown(options=sorted(df['petitioner'].unique()), description='Petitioner:')
respondent_widget = widgets.Dropdown(options=sorted(df['respondent'].unique()), description='Respondent:')
lcDispositionDirection_widget = widgets.Dropdown(options=sorted(df['lcDispositionDirection'].unique()), description='LC Disposition Direction:')
declarationUncon_widget = widgets.Dropdown(options=sorted(df['declarationUncon'].unique()), description='Declaration Uncon:')

# Create a button to trigger prediction
predict_button = widgets.Button(description='Predict')

# Create an output widget to display the prediction
output_widget = widgets.Output()

# Function to handle button click event
def on_predict_button_clicked(b):
    with output_widget:
        clear_output()
        input_data = {
            'term': term_widget.value,
            'caseOrigin': caseOrigin_widget.value,
            'issueArea': issueArea_widget.value,
            'petitioner': petitioner_widget.value,
            'respondent': respondent_widget.value,
            'lcDispositionDirection': lcDispositionDirection_widget.value,
            'declarationUncon': declarationUncon_widget.value
        }

        # Convert input data to a DataFrame
        input_df = pd.DataFrame([input_data])

        # Make prediction using the trained model
        prediction = model.predict(input_df)[0]

        # Display the prediction
        if prediction == 0:
            print("Denied")
        elif prediction == 1:
            print("Affirmed")


# Assign the function to the button's on_click event
predict_button.on_click(on_predict_button_clicked)


# Display the widgets
display(term_widget, caseOrigin_widget, issueArea_widget, petitioner_widget, respondent_widget,
        lcDispositionDirection_widget, declarationUncon_widget, predict_button, output_widget)



Dropdown(description='Term:', options=(1946, 1947, 1948, 1949, 1950, 1951, 1952, 1953, 1954, 1955, 1956, 1957,…

Dropdown(description='Case Origin:', options=(1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 15.0, 16.0, 1…

Dropdown(description='Issue Area:', options=(1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, na…

Dropdown(description='Petitioner:', options=(1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13…

Dropdown(description='Respondent:', options=(1.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 16.0, 19.0, 21.0, 25.0, 26.0, …

Dropdown(description='LC Disposition Direction:', options=(1.0, 2.0, 3.0, nan), value=1.0)

Dropdown(description='Declaration Uncon:', options=(1.0, 2.0, 3.0, 4.0, nan), value=1.0)

Button(description='Predict', style=ButtonStyle())

Output()